In [4]:
import numpy as np

## 31 - Coin Sums

<p>In the United Kingdom the currency is made up of pound (£) and pence (p). There are eight coins in general circulation:</p>
<blockquote>1p, 2p, 5p, 10p, 20p, 50p, £1 (100p), and £2 (200p).</blockquote>
<p>It is possible to make £2 in the following way:</p>
<blockquote>1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p</blockquote>
<p>How many different ways can £2 be made using any number of coins?</p>
coins?

In [5]:
coins = [200, 100, 50, 20, 10, 5, 2, 1]
target = 200
#try a recursive approach
def combinations(target, coins):
    if target == 0: # if zero, there are no ways, so return list of 0 combinations
        return [[]]
    if target < 0 or not coins: # if under target or no coins left, return empty list with no combinations
        return []
    
    #we either use the current coin in reaching the target, or we do not, so we run two recursive scenarios:
    #Combinations of using the current coin:
    combinations_with_current_coin = combinations(target - coins[0], coins)
    
    #combinations without using the current coin
    combinations_without_current_coin = combinations(target, coins[1:])
    
    #add the current coin to all combinations that make use of it
    all_combinations = [[coins[0]] + combi for combi in combinations_with_current_coin]
    
    #finally return all combinations with current coin, and the ones without the current coin
    return all_combinations + combinations_without_current_coin

In [6]:
%%time
result = combinations(target, coins)
print(len(result))

73682
CPU times: user 2.63 s, sys: 50.3 ms, total: 2.68 s
Wall time: 2.68 s


In [7]:
%%time
#What if we try to look at all ways to make combinations for all values up to 200 pennies?
#There is one way to make 0, by using no coins. There is 1 way to make 1, by using 1 penny.
#There are two ways to make 2, by using 2 pennies and by using 1 2 penny.

#empty array to store combinations for all values up to 200, including 0 is 201 long
combinations = np.zeros(201, dtype=np.int32)
coins = [1,2,5,10,20,50,100,200]
#start with combination for value 0, which is one. By using no coins.
combinations[0] = 1

#for every coin
for coin in coins:
    # A coin value has one way to contribute to creating exactly its own value, but cant contribute to creating lower values. 
    # Therefore starting from coin-value and up to target;
    for i in range(coin, 201):
        # update how many ways we can reach the given value
        # with the given coin and all previous coins combined. 
        # by adding the current number of combinations to the amount of ways we got to the value minus the current coin. 
        # for example there are for value 8 given coin 5, we need to add all the ways to make 3 from 1 and 2,
        # plus all the ways to make 8 from 1 and 2 to get all the ways we can make 8 from 1, 2 and 5. Therefore 
        # we can iteratively add the number of current combinations 
        # to the number of combinations minus the current coin value:
        combinations[i] += combinations[i-coin]
#we want to know how many ways there are to make 200:
print(combinations[-1])

73682
CPU times: user 0 ns, sys: 426 µs, total: 426 µs
Wall time: 429 µs


## 32 - Pandigital Products

<p>We shall say that an $n$-digit number is pandigital if it makes use of all the digits $1$ to $n$ exactly once; for example, the $5$-digit number, $15234$, is $1$ through $5$ pandigital.</p>

<p>The product $7254$ is unusual, as the identity, $39 \times 186 = 7254$, containing multiplicand, multiplier, and product is $1$ through $9$ pandigital.</p>

<p>Find the sum of all products whose multiplicand/multiplier/product identity can be written as a $1$ through $9$ pandigital.</p>

<div class="note">HINT: Some products can be obtained in more than one way so be sure to only include it once in your sum.</div>


### start solution
Ok, lets not rush of to bruteforce all integers... There are some constraints here to check.

$a \times b = c$ and  ```len(a) + len(b) + len(c) == 9```

The length of a and b determine the length of c, so we can check some upper limits by figuring out how long the products are for ```len(a) = [1..9]```and ```len(b) == [1..9]``` So for ```len(a)==1``` and ```len(b)==1``` we get minimal $1 \times 2 = 2$ which has len 1, and maximally $9 \times 8 = 72$ which has len 2. For lens 2 and 1 we get $11 \times 2 = 22$ with len 2 and $87 \times 9 = 783$ with len 3. If we continue that manually because that is about as fast as writing the code for this step: 

| len |A=1|A=2|A=3|A=4|
|---|---|---|---|---|
|B=1| 1-2 | 2-3 | 3-4 | 4-5 |
|B=2| 2-3 | 3-4 | 4-5 | 5-6 |
|B=3| 3-4 | 4-5 | 5-6 | 6-7 |
|B=4| 4-5 | 5-6 | 6-7 | 7+  | 

Gives this little table of product lenghts, and the constraint that `len(c) == 9 - (len(a)+len(b))`, we see that c must a product of either a 1-digit and 4-digit number, or a product of a 2-digit and 3-digit number, because these are the only scenarios that can satisfy $a \times b = c$ and  ```len(a) + len(b) + len(c) == 9```.

So then we run two searches. One for A up to 9 and B from 1234 to 9876, and one for A up to 98 and b from 123 to 987. We`ll see if that is enough restriction to make chekcing pandigitalism quick enough or if we have to do more thinking...


In [8]:
def check_pandigital_product(a, b):
    c = a*b
    #this check feels silly
    if "".join(sorted(str(a)+str(b)+str(c))) != "123456789":
        return False, c
    return True, c

In [9]:
check_pandigital_product(39, 186)

(True, 7254)

In [10]:
check_pandigital_product(15, 29)

(False, 435)

In [11]:
#gather the products
pandigital_products = []
#define the search ranges
search_one = (range(1, 10), range(123, 9877))
search_two = (range(11, 99), range(123, 988))
for range_a, range_b in [search_one, search_two]:
    for a in range_a:
        for b in range_b:
            #check for pandigitalism, add to list of not seen before
            is_pandigital, c = check_pandigital_product(a, b)
            if is_pandigital and c not in pandigital_products:
                pandigital_products.append(c)
#print the answer
print(f"The sum of all 1 through 9 pandigital products = {sum(pandigital_products)}")
    

The sum of all 1 through 9 pandigital products = 45228


## 33 - Digital Cancelling Fractions

<p>The fraction $49/98$ is a curious fraction, as an inexperienced mathematician in attempting to simplify it may incorrectly believe that $49/98 = 4/8$, which is correct, is obtained by cancelling the $9$s.</p>
<p>We shall consider fractions like, $30/50 = 3/5$, to be trivial examples.</p>
<p>There are exactly four non-trivial examples of this type of fraction, less than one in value, and containing two digits in the numerator and denominator.</p>
<p>If the product of these four fractions is given in its lowest common terms, find the value of the denominator.</p>


In [12]:
fracs = [] # gather our found non-trivial fractions
for a in range(11, 100): #for all values with two digits
    for b in range(a+1, 100): #for all values with two digits higher than a (to only check fractions smaller than 1)
        string_a, string_b = str(a), str(b) #strings to check if digits in both numbers
        if (
            #if value in both numbers
            (string_a[0] in string_b or string_a[1] in string_b) and
            #if there is no 0 in the first number
            ("0" not in string_a) and 
            # if the numbers arent two of the same digit
            (string_a[0] != string_a[1] and string_b[0] != string_b[1])
        ):
            frac = a/b #calculate fraction
            set_a = set([int(x) for x in string_a]) 
            set_b = set([int(x) for x in string_b])
            digits = list(set_a.difference(set_b).union(set_b.difference(set_a))) #find the numbers that the sets dont have in common
            
            if len(digits) == 2 and not 0 in digits: #if there are exactly two numbers in the set of digits, and they are both not zero:
                digits = [[digit for digit in digits if digit in set_a][0], [digit for digit in digits if digit in set_b][0]] #sort so that the first digit belongs to number a and the second digit belongs to number b
                simplified = digits[0]/digits[1] # calculate simplified fraction
                if frac == simplified: # if the frac happens to have the same value as the simplified fraction we have found a non-trivial example
                    fracs.append((a,b))
           
print(f"Our fractions are:")
for frac in fracs:
    print(f" {frac[0]}/{frac[1]}")

Our fractions are:
 16/64
 19/95
 26/65
 49/98


In [13]:
# put our fractions in a numpy array to more easily multiply them
fracs = np.vstack(fracs)

In [14]:
#the product of our fractions is num/denom:
num, denom = np.product(fracs, axis = 0)

In [15]:
num, denom

(387296, 38729600)

In [16]:
#to simplify this fraction we have to find common denominators, which we can do recursively. 
#I am using recursion too much these days. 
def get_common_denom(a, b):
    if b == 0:
        return a
    return get_common_denom(b, a%b)

In [17]:
common_denom = get_common_denom(num, denom)

In [18]:
num//=common_denom
denom//=common_denom

In [19]:
print(f"The final fraction is {num}/{denom}, which makes the answer {denom}")

The final fraction is 1/100, which makes the answer 100


Wild that that would all add up to be 100....

## 34 - Digit Factorials

<p>$145$ is a curious number, as $1! + 4! + 5! = 1 + 24 + 120 = 145$.</p>
<p>Find the sum of all numbers which are equal to the sum of the factorial of their digits.</p>
<p class="smaller">Note: As $1! = 1$ and $2! = 2$ are not sums they are not included.</p>


First we have to find an upper limit? what is the largest number where the sum of the factorials of the digits is still smaller than or equal to the number? Is there such a limit? 99 is going to be much more problematic than 211 for example. 

We can use the same logic we used for problem 30 where we said that since 

In [77]:
import math

In [78]:
def digit_factorial_sum(number):
    return np.sum([math.factorial(int(x)) for x in str(number)])

In [79]:
for i in range(1, 10):
    for j in range(9, 2, -1):
        upper_limit = int(str(j)*i)
        lensum = len(str(digit_factorial_sum(upper_limit)))
        if i > lensum:
            print(f"The highest {i}-digit number, {upper_limit}, has a digit facorial sum of length {lensum}")
            break #we found the upper limit

The highest 3-digit number, 444, has a digit facorial sum of length 2
The highest 4-digit number, 5555, has a digit facorial sum of length 3
The highest 5-digit number, 66666, has a digit facorial sum of length 4
The highest 6-digit number, 777777, has a digit facorial sum of length 5
The highest 7-digit number, 8888888, has a digit facorial sum of length 6
The highest 8-digit number, 99999999, has a digit facorial sum of length 7
The highest 9-digit number, 999999999, has a digit facorial sum of length 7


this means that our upper limit is the highest 7 digit number:

In [94]:
upper_limit = int("6"*5)

In [95]:
upper_limit

66666

In [96]:
def does_it_sum(number):
    return number == digit_factorial_sum(number)

In [97]:
does_it_sum(145)

True

In [98]:
print(f"Finding all numbers from 3 to {upper_limit} where the digit factorial sum is the same as the number")

Finding all numbers from 3 to 66666 where the digit factorial sum is the same as the number


In [99]:
%%time
stupid_solution = np.sum([x for x in range(3, upper_limit) if does_it_sum(x)])

CPU times: user 277 ms, sys: 9.86 ms, total: 287 ms
Wall time: 286 ms


In [100]:
stupid_solution

40730

In [ ]:
#first we have to figure out an upper limit? What is the largest number where the sum of the fac

## 35 - Circular Primes

<p>The number, $197$, is called a circular prime because all rotations of the digits: $197$, $971$, and $719$, are themselves prime.</p>
<p>There are thirteen such primes below $100$: $2, 3, 5, 7, 11, 13, 17, 31, 37, 71, 73, 79$, and $97$.</p>
<p>How many circular primes are there below one million?</p>


In [35]:
def rotate(seq: int):
    
    seq = str(seq)
    l = len(seq)
    seq+=seq
    seqs = [int(seq[i:i+l]) for i in range(l)]        
    return seqs
        
    

In [36]:
rotate(917)

[917, 179, 791]

In [37]:
from utils import is_prime
is_prime_array = np.vectorize(is_prime) #so that I can apply the function along an array
numbers = np.arange(1, 1e6, dtype = int)
primes = numbers[is_prime_array(numbers)] #get all primes

In [38]:
def is_circular(prime):
    rotations = rotate(prime)
    for p in rotations:
        if not is_prime(p):
            return False
    return True
            

In [39]:
circulars = [p for p in primes if is_circular(p)]
    

In [40]:
len(circulars)

55

In [41]:
circulars[-1]

999331

In [42]:
circulars[:13]

[2, 3, 5, 7, 11, 13, 17, 31, 37, 71, 73, 79, 97]